<a href="https://colab.research.google.com/github/TorbjornLarsson/SCDA/blob/main/Lab2_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab exercise 2: Regression, Least squares, QR

## Part 3: Under the Hood

_In Part 2 of the lab, we did the curve fitting using **Polynomial.fit**. But
what is happening inside this function? What numerical methods are used under the
hood? In this part of the lab, you will solve the same problem as in Part 2, but explicitly set up the problem and
use the numerical methods that are built-in to **Polynomial.fit**._

### Short background:

Assume you have a dataset $x = \{x_1, x_2, \cdots , x_n\}$ and $y = \{y_1, y_2, \cdots , y_n\}$. For example, this could correspond to the years x and corresponding average annual temperatures y in your dataset from the previous part of the lab. Let's say that you are supposed to fit a straight line, a 1st degree polynomial, to the data. This means that we would like the data to (roughly) conform to $p(x) = a+bx$. We say that we use this polynomial as the _anzats_, and the task is to find the coefficients $a$ and $b$ such that $p(x)$ is "as close to the data" as possible.

If the data would fit perfectly, we would find an $a$ and $b$ such that:

$$\begin{eqnarray}
a + bx_1  &=& y_1 \\
a+ bx_2  &=& y_2 \\
\vdots \\
a+bx_n &=& y_n \\
\end{eqnarray}$$

This sytem of equations can be written equivalently in matrix form:

$$\begin{pmatrix} 1&x_1  \\ 1&x_2 \\ \vdots & \vdots \\ 1& x_n \end{pmatrix} \begin{pmatrix} a \\ b \end{pmatrix} = {} \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{pmatrix}$$


Hence, we get a linear equation system $Av = y$ to solve, and the coefficients in $v$ is the
solution. The system is, however, overdetermined (more rows than columns ⇔ more equations
than unknowns), so there is no exact solution. This is due to the fact that there is no straight line
that can pass through each of the data points.
The solution to this is to solve the so-called normal equations:

$$A^TAv = A^Ty$$

Here, $A^T$ stands for “$A$-transpose”. Solving the normal equations for $v$ will give us the polynomial closest to the data. This is also equivalent to
finding the minimum

$$\min_{v} ‖Av-y‖_2$$

This minimization problem is also called _the least squares problem_. Why this is the case and how the normal equations are derived will be explained in the lecture corresponding to this module.

### To do:

Try out the procedure described above, for the temperature data in Uppsala during the period
1960-2020, in the file **tempAverage1960_2020.txt**. Do this by following the steps below:

_1) Import the required packages and load each column in the dataset into numpy arrays the same way you did in Part 2._

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load the new dataset
data = np.loadtxt('tempAverage1960_2020.txt', usecols=(0,1))
years = data[:, 0]
temperatures = data[:, 1]



_2) Set up the matrix $A$. You can see the structure of $A$ above: the first column in $A$ will be the first column in the data set (the year data), and
the second column in $A$ will just be a column with ones. Also create the right-hand-side
$y$ (which here is the second column in the dataset, the temperatures)._

**Hint 1:** You can create a $(N\times 2)$-matrix initialized with ones using the command `numpy.ones((N, 2))`. Here, N is the number of rows in the matrix, which is equal to the number of datapoints. You can find the number of datapoints using `len(x)`, where x is one of the columns you loaded in Question 1).

**Hint 2:** When you have created a $(N\times 2)$-matrix $A$ filled with ones, you can replace the second column in the matrix with the array, called for example _array_, you used to store the year-data in Question 1). You can do this with the command: $A[:,1] = array$, where $A[:,1]$ picks out the second column in $A$.

In [4]:
N = len(years)
A = np.ones((N, 2))
A[:, 1] = years
y = temperatures

print("Shape of A:", A.shape)
print("Shape of y:", y.shape)
print("First 5 rows of A:\n", A[:5])
print("First 5 elements of y:\n", y[:5])

Shape of A: (61, 2)
Shape of y: (61,)
First 5 rows of A:
 [[1.000e+00 1.960e+03]
 [1.000e+00 1.961e+03]
 [1.000e+00 1.962e+03]
 [1.000e+00 1.963e+03]
 [1.000e+00 1.964e+03]]
First 5 elements of y:
 [5.33114754 6.19232877 4.44027397 4.7090411  5.38360656]


_3) Compute $A^TA$ and $A^Ty$ and store the results in new variables. What size will the matrix $A^TA$ be of? How many equations and unknowns will the equation system $A^TAv = A^Ty$ have? Try to figure it out without checking in Python, and then check if you were right by printing the shape of the matrices._

**Hint 1:** You can compute the matrix transpose $A^T$ using the command `np.transpose(A)`.

**Hint 2:** In Python, you can multiply two matrices $A$ and $B$ (of appropriate dimensions) using the commands `A@B` or `np.matmul(A, B)`.

In [5]:
ATA = np.transpose(A) @ A
ATy = np.transpose(A) @ y

print("Shape of A^TA:", ATA.shape)
print("Shape of A^Ty:", ATy.shape)
print("A^TA:\n", ATA)
print("A^Ty:\n", ATy)

Shape of A^TA: (2, 2)
Shape of A^Ty: (2,)
A^TA:
 [[6.1000000e+01 1.2139000e+05]
 [1.2139000e+05 2.4158501e+08]]
A^Ty:
 [3.59863173e+02 7.16897951e+05]


_4) Next, we will solve the system of equations, the normal equations, in Python. To do this, we can use the function `np.linalg.solve()`. To solve a system of linear equations of the form $Ax = b$ using this function, we can call
`x = np.linalg.solve(A, b)`. Use this function to solve the normal equations, and print the coefficients. Compare the coefficients to the result you got in Part 2._

In [6]:
coefficients = np.linalg.solve(ATA, ATy)
print("Koefficienter (a, b):", coefficients)

Koefficienter (a, b): [-7.51567244e+01  4.07317189e-02]


_5a) Repeat the steps 2-4, but now fit a second order polynomial to the data. A second order
polynomial has the structure:_ $$p(x) = a_0+ a_1x+ a_2x^2$$

_Use this as the new anzats. Figure out how to construct the matrix and form a new matrix which you can call $B$_.

Hint: You can obtain the elementwise square of a vector x using the command `np.square(x)`.

_5b) Once you have the matrix, form the normal equations and solve the equation system. Compare the coefficients with the result you got in Part 2._

In [7]:
# 5a) Construct the matrix B for a second order polynomial
N = len(years)
B = np.ones((N, 3))
B[:, 1] = years
B[:, 2] = np.square(years)

# The right-hand-side y remains the same (temperatures)
y = temperatures

print("Shape of B:", B.shape)
print("First 5 rows of B:\n", B[:5])

# 5b) Form the normal equations and solve the equation system
BTB = np.transpose(B) @ B
BTy = np.transpose(B) @ y

print("\nShape of B^TB:", BTB.shape)
print("Shape of B^Ty:", BTy.shape)

coefficients_second_order = np.linalg.solve(BTB, BTy)
print("\nKoefficienter för andraordningens polynom (a0, a1, a2):", coefficients_second_order)

Shape of B: (61, 3)
First 5 rows of B:
 [[1.000000e+00 1.960000e+03 3.841600e+06]
 [1.000000e+00 1.961000e+03 3.845521e+06]
 [1.000000e+00 1.962000e+03 3.849444e+06]
 [1.000000e+00 1.963000e+03 3.853369e+06]
 [1.000000e+00 1.964000e+03 3.857296e+06]]

Shape of B^TB: (3, 3)
Shape of B^Ty: (3,)

Koefficienter för andraordningens polynom (a0, a1, a2): [ 1.91625401e+03 -1.96084279e+00  5.02908169e-04]


One issue with the normal equations is the so-called condition number. A high matrix
condition number leads to loss of accuracy when solving the equation system. If the
equation system is $Ax = b$, the relation between the condition number and the relative
error in $x$ and $b$ is:

**(relative error in solution $x$) $\leq$ cond($A$) (relative error in in-data $b$)**

Hence, the condition number of the matrix, cond($A$), might magnify the error in in-data
when solving the equation system. Remember that the smallest relative error in $b$ is $\approx$
$10^{-16}$ due to roundoff (but it can be much bigger if it is based on measurements).

_6) Calculate cond($A^TA$) and cond($B^TB$), respectively. Print the results. If the relative error in $b$
is $10^{-16}$, what error might we get in the solution here in the worst case?_

Hint: You can calculate the condition number of a matrix A using the command `np.linalg.cond(A)`.

In [8]:
cond_ATA = np.linalg.cond(ATA)
cond_BTB = np.linalg.cond(BTB)

print("Konditionstal för A^TA:", cond_ATA)
print("Konditionstal för B^TB:", cond_BTB)

Konditionstal för A^TA: 50596307121.39148
Konditionstal för B^TB: 4.297624312018623e+21


For the problem A, the relative error could be O(10^-6), but for B O(10^5).

The bad condition number in the normal equations is a serious problem, and usually
different modifications is introduced to improve the situation. More on this and the
condition number on the lecture!

**Shifting and scaling**

_7a) Do a first-degree polynomial fit, but now use the anzats:_ $$p(x) = a + b(x-\bar{x})$$

_Here, $\bar{x}$ is the mean of the x-values. Form the matrix $C$ and $C^TC$._

_7b) Compute cond($C^TC$) and
compare with cond($A^TA$). What happened?_

Apply the same fitting using the ansatz
$$p(x) = a+b\left(\frac{x-\bar{x}}{\sigma}\right) $$
and answer the same quesions.

In [9]:
# 7a) Do a first-degree polynomial fit, but now use the anzats:
# p(x)=a+b(x−x¯)
# Here, x¯ is the mean of the x-values. Form the matrix C and CTC .

x_bar = np.mean(years)

N = len(years)
C = np.ones((N, 2))
C[:, 1] = years - x_bar

CTC = np.transpose(C) @ C

# 7b) Compute cond( C^T C ) and compare with cond( A^T A ). What happened?
cond_CTC = np.linalg.cond(CTC)

print("Konditionstal för A^TA:", cond_ATA)
print("Konditionstal för C^TC (skiftad x-data):", cond_CTC)

# Apply the same fitting using the ansatz p(x) = a+b((x-x_bar)/sigma)
sigma = np.std(years)
C_scaled = np.ones((N, 2))
C_scaled[:, 1] = (years - x_bar) / sigma

CTC_scaled = np.transpose(C_scaled) @ C_scaled
cond_CTC_scaled = np.linalg.cond(CTC_scaled)

print("Konditionstal för C^TC (skiftad och skalad x-data):", cond_CTC_scaled)

Konditionstal för A^TA: 50596307121.39148
Konditionstal för C^TC (skiftad x-data): 310.0
Konditionstal för C^TC (skiftad och skalad x-data): 1.0


This is called scaling and is used in `numpy.Polynomial.fit` to improve the condition
number.